# 信息检索与数据挖掘实验一

##                      ——布尔查询模型
### 数据18.1 201800130083 刁龙飞

### 建立倒排索引

In [1]:
from nltk.stem.porter import PorterStemmer

In [2]:
#数据读取
def ReadData(text):
    data = []
    for line in text.readlines():
        data.append([eval(line).get('tweetId'),eval(line).get('text')]) #读取使用的tweetId和text两项
    return data

In [3]:
#数据预处理
def DataPreprocess(data_lists): #参数是一个二维列表，每一项表示为[id,text]
    file = []    
    unused_words_file = open('unused.txt','r')
    unused_words = unused_words_file.read()
    unused_words_file.close()
    unused_words_list = unused_words.split()
    for data_list in data_lists: 
        #全部变成小写
        data_list[1] = data_list[1].lower()
        # 特殊符号替换：可以保留一部分网址、邮箱等特殊词项
        # 删除所有的逗号、括号、其他特殊符号
        for ch in '"#$%&()*+,-—–;<=>[]^_‘{|}~“”""':
            data_list[1] = data_list[1].replace(ch,' ')         
        for s in ['…','..','...','....','.....','......','. ',' .',': ',"' "," '","''"]:            
            data_list[1] = data_list[1].replace(s,' ')
            
        # 特殊位置替换
        words_list = data_list[1].split()
        for j,item in enumerate(words_list):
            if item[0] in "?'./!":
                words_list[j] = item.replace(item[0],'')
            if item[-1] in "?'./!":
                words_list[j] = item.replace(item[-1],'')
        data_list[1] = ' '.join(words_list)
        t = data_list[1].split(' ')
        t.insert(0,data_list[0])
        file.append(t)
        
    # 去除停用词
    for i,words in enumerate(file):
        for j in unused_words_list:
            if j in words:
                words.pop(words.index(j))                    
    # 词项归一化，暂时不作处理
    
    # 词干还原 
    porter_stemmer = PorterStemmer()  
    for i,line in enumerate(file):
        for j,word in enumerate(line):
            line[j] = porter_stemmer.stem(word)
        file[i] = line  
          
    file.sort() #好像没什么用 原本的tweets就是有序的
    return file

In [4]:
#建立倒排索引
def get_inverted_index(file):
    inverted_index = {}
    tweet_dicts_list = []
    #建立每个tweet的词项字典
    for tweet in file:
        tweet[0] = eval(tweet[0]) #因为前面要过词干提取，所以那时候id要用字符串
        tweet_dict = {}
        for i,item in enumerate(tweet):
            if i==0:
                continue
            else:  #删掉空的词项
                if item == '':
                    continue
                tweet_dict[item] = [1,tweet[0]] 
        tweet_dicts_list.append(tweet_dict)
    #对词项字典进行合并
    for tweet_dict in tweet_dicts_list:
        for key in tweet_dict:    
            if key not in inverted_index:
                inverted_index[key] = tweet_dict[key]
            else:
                inverted_index[key].append(tweet_dict[key][1])
                inverted_index[key][0] +=1
    #对输出的倒排索引排序 根据key的字母序
    sorted_inverted_index_keys = sorted(inverted_index.keys())
    sorted_inverted_index_list = []
    for key in sorted_inverted_index_keys:        
        sorted_inverted_index_list.append([key,inverted_index[key]])  
    return sorted_inverted_index_list

In [5]:
tweets_open = open('tweets.txt','r')
data_ori = ReadData(tweets_open)
tweets_open.close()
print(data_ori[:10]) #读取的原始数据

[['28965792812892160', 'House may kill Arizona-style immigration bill, Rep. Rick Rand says: The House is unlikely to pass the "Ari... http://tinyurl.com/4jrjcdz.'], ['28967095878287360', "Mourners recall Sarge Shriver's charity, idealism \n    (AP): AP - R. Sargent Shriver was always an optimist, pio... http://bit.ly/gqMcdG"], ['28967672074993664', 'Bass Fishing Techniques: 2 Fantastic Tips To Improve Your Casting Skills'], ['28967914417688576', '#Financial Aid | Proper Method Of Getting Financial Aid For Education http://ping.fm/BK0R3 #applying-for-financial-aid financial-aid-essay #'], ['28968479176531969', "Supreme Court: NASA's intrusive background checks OK http://bit.ly/h2jgy9"], ['28968581949558787', 'The McDonalds music to fireworks is an all time low.'], ['28969422056071169', "@alyce Very sweet and quiet, if not polished - Bono & Hansard at Sgt Shriver's funeral 2day: http://youtu.be/Bf14XBbcVZg (when was ...cont'd"], ['28971749961891840', "So, Avon&Somerset Police have charge

In [6]:
id_list_file = open('all_text.txt','w',encoding='utf-8')
id_list_file.write(str(data_ori))
id_list_file.close()
data = DataPreprocess(data_ori)
print(data[:10]) #预处理后的数据

[['28965792812892160', 'hous', 'may', 'kill', 'arizona', 'style', 'immigr', 'bill', 'rep', 'rick', 'rand', 'say', 'hous', 'unlik', 'pass', 'the', 'ari', 'http://tinyurlcom/4jrjcdz'], ['28967095878287360', 'mourner', 'recal', 'sarg', "shriver'", 'chariti', 'ideal', 'ap', 'ap', 'r', 'sargent', 'shriver', 'alway', 'optimist', 'pio', 'http://bit.ly/gqmcdg'], ['28967672074993664', 'bass', 'fish', 'techniqu', '2', 'fantast', 'tip', 'improv', 'your', 'cast', 'skill'], ['28967914417688576', 'financi', 'aid', 'proper', 'method', 'get', 'financi', 'aid', 'educ', 'http://ping.fm/bk0r3', 'appli', 'for', 'financi', 'aid', 'financi', 'aid', 'essay'], ['28968479176531969', 'suprem', 'court', "nasa'", 'intrus', 'background', 'check', 'ok', 'http://bit.ly/h2jgy9'], ['28968581949558787', 'mcdonald', 'music', 'firework', 'all', 'time', 'low'], ['28969422056071169', '@alyc', 'veri', 'sweet', 'quiet', 'if', 'not', 'polish', 'bono', 'hansard', 'sgt', "shriver'", 'funer', '2day', 'http://youtu.be/bf14xbbcvzg

In [7]:
inverted_index = get_inverted_index(data)
inverted_index_file = open('inverted_index.txt','w',encoding='utf-8')
inverted_index_file.write(str(inverted_index))
inverted_index_file.close()
print(inverted_index[12010:12020])

[['em', [24, 30140006722969600, 301710958420688896, 301835432755331072, 302137452007669761, 302455455777701890, 30280495124193280, 304397288481103872, 305444522333200385, 30637444080607233, 307544274776510464, 309964690166001664, 311966509855547392, 313423107731890176, 33539217275486208, 33840094867623936, 34314860862898178, 34397493319962624, 624249890965581824, 624790327031918593, 624935953241649152, 624937249294168064, 625776810563923968, 626217304729235456, 626430736099373056]], ['email', [22, 29576869535817728, 29898299192385538, 303271814136729600, 303646713581826049, 307894083920207872, 307894599823802369, 307898622173908992, 307902648680599552, 307904263512809475, 307912480133361664, 307914334019936256, 307936073126907907, 307937528521060352, 308261559489163265, 308321307328667648, 308333579899908096, 308367981560356866, 309080413454876672, 310751122170212352, 31771605050855424, 623890778826235904, 625014424483561472]], ['eman', [2, 30067826060435456, 30067834360963073]], ['ema

### 根据倒排索引建立布尔查询模型

In [8]:
#读入ID
id_open = open('all_text.txt','r',encoding='utf-8')
id_data = id_open.read()
id_open.close()
id_list = []
for i in eval(id_data):
    id_list.append(eval(i[0]))

In [9]:
'''
#读入倒排索引
inverted_index_open = open('inverted_index.txt','r',encoding='utf-8')
inverted_index_str = inverted_index_open.read()
inverted_index_open.close()
inverted_index = eval(inverted_index_str)
'''

"\n#读入倒排索引\ninverted_index_open = open('inverted_index.txt','r',encoding='utf-8')\ninverted_index_str = inverted_index_open.read()\ninverted_index_open.close()\ninverted_index = eval(inverted_index_str)\n"

In [10]:
def query_preprocess(query): #输入为一个字符串 采用大致相同的预处理方式 不考虑停用词
    #全部变成小写
    query = query.lower()
    #整句特殊符号的替换
    for ch in '!"#$%&()*+,-—–;<=>[]^_‘{|}~“”""':
        query = query.replace(ch, " ")   
    for s in ['…','...','....','.....','......','. ',' .',': ',"' "," '","''"]:            
        query = query.replace(s,' ')
    query_word_list = query.split()     
    # 特殊位置替换
    for j,item in enumerate(query_word_list):
        if item[0] in "?'./":
            query_word_list[j] = item.replace(item[0],'')
        if item[-1] in "?'./":
            query_word_list[j] = item.replace(item[-1],'')
            
    # 去除停用词
                   
    # 词项归一化，暂时不作处理
    
    # 词干还原
    porter_stemmer = PorterStemmer()  
    for i,word in enumerate(query_word_list):
        query_word_list[i] = porter_stemmer.stem(word) 
    logical_operator_list = ['and','or']     
    for i,word in enumerate(query_word_list):
        if word in logical_operator_list:
            query_word_1 = ' '.join(query_word_list[:i])
            query_word_2 = ' '.join(query_word_list[i+1:])
            logical_operator = word
            return [logical_operator,query_word_1,query_word_2]
        #输出根据输入的不同 分为两种
    return [''.join(query_word_list)]  

In [11]:
#示例
print(query_preprocess('ron and birthday'))
print(query_preprocess('NOT Ron or birthday'))
print(query_preprocess('HARRY POTTER'))

['and', 'ron', 'birthday']
['or', 'not ron', 'birthday']
['harripotter']


In [12]:
def list_op(list1,list2,op): #求两个有序列表的合并算法 交集或补集
    answer = []
    if op == 'and':
        i = j = 0
        while i<len(list1) and j<len(list2):
            if list1[i]==list2[j]:
                answer.append(list1[i])
                i+=1
                j+=1
            elif list1[i]<list2[j]:
                i+=1
            elif list1[i]>list2[j]:
                j+=1
    elif op == 'or':
        answer = list1[:] #要新建不要索引
        for j in list2:
            if j not in answer:
                answer.append(j)
        answer.sort()   
    return answer

In [13]:
a = [2,3,6,9,10,11]
b = [1,2,6,7,9,10,11,13]
print(list_op(a,b,'and'))
print(list_op(a,b,'or'))

[2, 6, 9, 10, 11]
[1, 2, 3, 6, 7, 9, 10, 11, 13]


In [14]:
#最简单的基于一个单词的查询
def basic_query(word,inverted_index):
    for index in inverted_index:
        if index[0] == word:
            return index[1][1:]
    return []

In [15]:
#简单的布尔查询
def simple_bool_query(query_word_1,query_word_2,logical_operator,inverted_index,all_id): 
    #字符串可能含有not;and/or;一个很复杂的多重列表；所有tweets的ID 用来求补
    result = []
    for i in [query_word_1,query_word_2]:
        query_word_list = i.split(' ')
        if len(query_word_list) == 2 and query_word_list[0]=='not':
            word = query_word_list[1]
            #从倒排索引中找到这个word对应的索引，然后取补
            for index in inverted_index:
                if index[0] == word:
                    result.append( list(set(all_id) - set(index[1][1:])) )
        elif len(query_word_list) == 1:
            #从倒排索引中找到这个word对应的索引
            word = query_word_list[0]
            for index in inverted_index:
                if index[0] == word:
                    result.append(list(index[1][1:]))
        else:
            word = ''.join(query_word_list)
            for index in inverted_index:
                if index[0] == word:
                    result.append(list(index[1][1:]))
    try:
        if logical_operator == 'and':
            result_id = list(set(result[0]) & set(result[1])) 
        #    result_id = list_op(result[0],result[1],'and')
        elif logical_operator == 'or':
            result_id = list(set(result[0]).union(result[1]))
        #    result_id = list_op(result[0],result[1],'or')
    except IndexError: #如果是不存在的查询结果 就会出现异常
        if result == []: #两个都不存在
            return []
        else: #一个不存在
            return result[0]
    return result_id

In [16]:
#复杂的布尔查询及查询优化
def complex_bool_query(simple_query_list,logical_operator_list,inverted_index,all_id):
    if logical_operator_list == ['and']*len(logical_operator_list):#查询优化
        optimized_query_list = []        
        for i in simple_query_list:
            if i[0]=='and':
                optimized_query_list.append(basic_query(i[1],inverted_index))
                optimized_query_list.append(basic_query(i[2],inverted_index))
            elif i[0]=='or':
                optimized_query_list.append(simple_bool_query(i[1],i[2],'or',inverted_index,all_id))
            else:
                optimized_query_list.append(basic_query(i[0],inverted_index))
        optimized_query_list.sort(key = lambda x:len(x))
        for i in optimized_query_list:
            optimized_query_list[0] = list_op(optimized_query_list[0],i,'and')
        return optimized_query_list[0]
    else:
        #首先完成每一个简单查询
        temp_query_list = []
        for i in simple_query_list:
            if len(i) == 3:
                temp_query_list.append(simple_bool_query(i[1],i[2],i[0],inverted_index,all_id))
            elif len(i) == 1:
                temp_query_list.append(basic_query(i[0],inverted_index))     
        #从左到右执行逻辑查询
        for i,op in enumerate(logical_operator_list):
            if i == 0:
                query_result_list = list_op(temp_query_list[i],temp_query_list[i+1],op)
            else:
                query_result_list = list_op(query_result_list,temp_query_list[i+1],op)
        return query_result_list

In [17]:
def print_tweets(qualified_id_list, id_data):
    qualified_quary_list = []
    for i in eval(id_data):
            if eval(i[0]) in qualified_id_list:
                qualified_quary_list.append([i[1]])
    print('共找到符合条件的结果' + str(len(qualified_quary_list)) + '条')
    for i in qualified_quary_list:
        print(i)
    qualified_quary_list.clear()
    return  

In [18]:
#简单查询
query = input(' \n 请输入您的查询语句：')
query_list = query_preprocess(query)
qualified_quary_list = []
if len(query_list) == 1:   #简单查询
    qualified_id_list = basic_query(query_list[0],inverted_index)
    print_tweets(qualified_id_list, id_data)    
elif len(query_list) == 3:    #简单布尔查询
    qualified_id_list = simple_bool_query(query_list[1],query_list[2],query_list[0],inverted_index,id_list)   
    print_tweets(qualified_id_list, id_data)
else:
    print('输入异常！')

 
 请输入您的查询语句：hello
共找到符合条件的结果15条
["What Hides In @TacoBell ‘Beef’ http://bit.ly/fv2Fqh guess I ordered the Taco 'meat filling' instead of beef 2nite. Hello @DelTaco my friend!"]
['Hello from LAX..the TSA made my last few moments here horrible, otherwise a super trip. I will be back in a week but only for one night.']
['@Eli_Tanning Only for iced Coffee & Tazo Iced Teas. Say hello to Trenta  http://bit.ly/hzZDL5']
['Current TV just popped up on radar in a major way.  #Hello! http://huff.to/hF1Ib3 via']
['Bye Bye #USAirways - hello #AmericanAirlines: http://t.co/ACXOvGgR  the latest mega merger will create the worlds largest airline']
['hello worst nightmare...BBC News - Meteor strike injures hundreds in central Russia http://t.co/gl193lEV']
['Chinese cyber-attacks: Hello, Unit 61398 | The Economist http://t.co/TDU6Qq5E3T']
['Hello World, Cornell scientists 3D print ears with help from rat tails and cow ears:   Science! A t... http://t.co/eRZyjXVkTP Thank You!']
['hello Sherlock']
['VIDE

In [20]:
#高级查询
bool_query_num = int(input('请输入高级查询包含的简单查询数量：'))
simple_query_list = []
logical_operator_list = []
qualified_complex_id_list = []
for i in range(bool_query_num):
    simple_query_list.append(query_preprocess(input('请输入第{}个简单查询：'.format(i+1))))
    if i < bool_query_num-1:
        x = input('请选择第{}和{}个简单查询之间的逻辑连接词（1:and 2:or)'.format(i+1,i+2))
        if x=='1':
            logical_operator_list.append('and')
        elif x=='2':
            logical_operator_list.append('or')
        else:
            print('输入错误，默认为"and"')
            logical_operator_list.append('and')    
print(simple_query_list)
print(logical_operator_list)
qualified_complex_id_list = complex_bool_query(simple_query_list,logical_operator_list,inverted_index,id_list)
print_tweets(qualified_complex_id_list, id_data)

请输入高级查询包含的简单查询数量：3
请输入第1个简单查询：Ron and birthday
请选择第1和2个简单查询之间的逻辑连接词（1:and 2:or)1
请输入第2个简单查询：Boys or girls
请选择第2和3个简单查询之间的逻辑连接词（1:and 2:or)1
请输入第3个简单查询：Happy
[['and', 'ron', 'birthday'], ['or', 'boy', 'girl'], ['happi']]
['and', 'and']
共找到符合条件的结果1条
['Happy birthday to my boy Ron Weasley']
